In [1]:
import os
from shlex import join
import numpy as np
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium import utils
from gymnasium import *
import mujoco
import json
import pandas as pd
from sklearn.model_selection import train_test_split

import mujoco_viewer

In [2]:
datafile = 'dataset_soft/dataset_slider1.json'

with open(datafile) as f:
			data = json.load(f)

# df = pd.read_json(self.datafile)
df = pd.json_normalize(data['data'])
X = df.drop('actions', axis=1)
y = pd.DataFrame(df['actions'].to_list())  # Assuming 'actions' is a list
X_flat = X.apply(lambda x: pd.Series([i for _list in x for i in _list]), axis=1)
# y_flat = y.apply(lambda x: pd.Series([i for _list in x for i in _list]), axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.01, random_state=42)
f.close()

In [4]:
df = pd.json_normalize(data['data'])
X = df.drop('actions', axis=1)
y = pd.DataFrame(df['actions'].to_list())  # Assuming 'actions' is a list
ee_position = pd.DataFrame(X['end-effector-position'].tolist(), columns=['ee_pos_x', 'ee_pos_y', 'ee_pos_z'])
# swap z and x columns
# ee_position = ee_position[['ee_pos_z', 'ee_pos_y', 'ee_pos_x']]
X = pd.concat([X.drop('end-effector-position', axis=1), ee_position], axis=1)
ee_orientation = pd.DataFrame(X['end-effector-orientation'].tolist(), columns=['ee_ori_w', 'ee_ori_x', 'ee_ori_y', 'ee_ori_z'])
X = pd.concat([X.drop('end-effector-orientation', axis=1), ee_orientation], axis=1)
# y_flat = y.apply(lambda x: pd.Series([i for _list in x for i in _list]), axis=1)

In [6]:
X.head()

,end-effector-vel,end-effector-angular-velocity,disk11-pose,disk11-ori,disk12-pose,disk12-ori,disk13-pose,disk13-ori,disk21-pose,disk21-ori,...,disk33-pose,disk33-ori,tendon_lengths,ee_pos_x,ee_pos_y,ee_pos_z,ee_ori_w,ee_ori_x,ee_ori_y,ee_ori_z
0,"[0.02814370021224022, 0.018163999542593956, -0...","[-0.04424430802464485, 0.05046964809298515, 0....","[-0.002063153078779578, -0.0019974224269390106...","[0.9993481040000916, 0.025116004049777985, -0....","[-0.007888488471508026, -0.007641550153493881,...","[0.9977763891220093, 0.04638473317027092, -0.0...","[-0.01690080016851425, -0.01637839712202549, 0...","[0.9959816932678223, 0.06227249279618263, -0.0...","[-0.022067690268158913, -0.02169339545071125, ...","[0.9999876022338867, -0.0007600064272992313, 0...",...,"[0.06960684806108475, 0.06908269226551056, 0.6...","[0.9732527732849121, -0.17311331629753113, 0.1...","[0.40154579281806946, 0.40154579281806946, 0.4...",0.084189,0.086023,0.737381,0.973253,-0.173113,0.150910,0.006092
1,"[-0.06516364216804504, 0.09939750283956528, -0...","[2.1765053272247314, 0.7772970795631409, -0.12...","[-0.0004843173373956233, 0.0004039945779368281...","[0.9999604225158691, -0.005911596585065126, -0...","[-0.002125208033248782, 0.0019887180533260107,...","[0.999776303768158, -0.015148498117923737, -0....","[-0.005167542491108179, 0.005286411382257938, ...","[0.9998974204063416, -0.010622682049870491, -0...","[-0.008004960604012012, 0.008957884274423122, ...","[0.9997692704200745, -0.01843230240046978, -0....",...,"[-8.275821164716035e-05, 8.020362292882055e-05...","[0.9995248913764954, 0.029417503625154495, 0.0...","[0.4787539541721344, 0.4787539541721344, 0.478...",0.000820,-0.002861,0.771691,0.999525,0.029418,0.009076,-0.001478
2,"[-0.0016522167716175318, -0.000103084305010270...","[-1.776824951171875, -0.1704944670200348, 0.00...","[-1.8733482647803612e-05, -0.00032342967460863...","[0.9999896883964539, 0.004535334184765816, -0....","[-5.424190021585673e-05, -0.001485680812038481...","[0.9999438524246216, 0.010594635270535946, -0....","[-7.282019214471802e-05, -0.003710765158757567...","[0.9999414086341858, 0.01082631852477789, -0.0...","[1.273675820812059e-06, -0.006258692592382431,...","[0.9999029040336609, 0.013899064622819424, 0.0...",...,"[0.00019055626762565225, 0.0010340935550630093...","[0.9992915987968445, -0.037471361458301544, -0...","[0.45412254333496094, 0.45412254333496094, 0.4...",-0.000158,0.004779,0.770855,0.999292,-0.037471,-0.003483,0.000197
3,"[-0.03455907478928566, -0.007279515266418457, ...","[0.040709517896175385, 0.2243911325931549, 0.0...","[-0.024400439113378525, 0.00032290772651322186...","[0.9506271481513977, -0.0033306130208075047, -...","[-0.08495992422103882, 0.001753119402565062, 0...","[0.8444237112998962, -0.005140251014381647, -0...","[-0.1619933396577835, 0.0051006143912673, 0.14...","[0.7288240790367126, -0.0049835857935249805, -...","[-0.24098917841911316, 0.006648026406764984, 0...","[0.6114534735679626, 0.034547656774520874, -0....",...,"[-0.5711156129837036, -0.055171623826026917, -...","[0.4551008939743042, 0.11337997764348984, -0.8...","[0.5064986944198608, 0.5064986944198608, 0.506...",-0.610127,-0.067984,-0.101070,0.455101,0.113380,-0.878889,0.087074
4,"[0.03604229912161827, 0.02039589360356331, 0.0...","[0.22407041490077972, -0.0011421800591051579, ...","[0.005570658016949892, 0.02331455796957016, 0....","[0.9524502754211426, -0.29589372873306274, 0.0...","[0.018645815551280975, 0.0815998762845993, 0.1...","[0.8502443432807922, -0.5106816291809082, 0.12...","[0.03344351798295975, 0.15664105117321014, 0.1...","[0.7405356168746948, -0.6505954265594482, 0.16...","[0.05166124925017357, 0.23392604291439056, 0.1...","[0.6408424973487854, -0.7293840050697327, 0.23...",...,"[0.20961041748523712, 0.552814781665802, -0.02...","[0.4710836112499237, -0.8128296732902527, 0.33...","[0.44338977336883545, 0.44338977336883545, 0.4...",0.229231,0.589594,-0.056918,0.471084,-0.812830,0.339717,-0.044500


In [1]:
from model_arm_imitation import *

env = ArmEnv(datafile="dataset/data1.json", learn_imitation=True, render_mode=None)

env.reset()
env.step(env.action_space.sample())

2024-04-16 17:58:51.139390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 17:58:51.139420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 17:58:51.140519: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-16 17:58:51.145896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 17:58:51.776352: W tensorflow/compiler/tf2

({'observation': [-0.19050662194658113,
   0.1717622296246404,
   -0.025799694726646977,
   0.24954128225582267,
   -0.2128581525586526,
   -0.3083677451663038,
   -0.19050662194658113,
   0.1717622296246404,
   -0.025799694726646977,
   0.24954128225582267,
   -0.2128581525586526,
   -0.3083677451663038,
   0.16715245934801504,
   0.028501373233315575,
   2.335091305565319,
   -0.012862560632891717,
   0.9720637376137445,
   0.11920745507566918,
   -0.20178262364422,
   -0.265926389867174,
   0.5786448690961911,
   0.2,
   0.43821555255147326,
   0.19297266252204306,
   -0.7466597557810037,
   -0.4617660556149755]},
 -1.0987251641213034,
 False,
 False,
 {'reward_ee_pos': -2.246960177390422,
  'reward_ee_ori': 0.2791958990426946,
  'reward_action': 0.003887870345793263,
  'reward_ee_vel': 0.0001006024081877101,
  'del_ee_pos': 2.246960177390422,
  'del_ee_ori': 1.2758415964442908,
  'total_reward': -1.0987251641213034})

In [2]:
ee_pos = env.data.sensor("end-effector-pos").data
ee_ori = env.data.sensor("end-effector-orientation").data

base_pos = env.data.sensor("base-link-pos").data
base_ori = env.data.sensor("base-link-ori").data

print("ee_pos: ", ee_pos)
print("ee_ori: ", ee_ori)
print("base_pos: ", base_pos)
print("base_ori: ", base_ori)

ee_pos:  [0.04262926 0.01352108 2.33696309]
ee_ori:  [ 0.00490128  0.97943409  0.18993901 -0.06788241]
base_pos:  [5.06600e-02 3.46000e-04 1.12841e+00]
base_ori:  [ 0.41354962  0.41836919 -0.57493859  0.5686735 ]


In [7]:
# def quaternion_to_rotation_matrix(quaternion):
#     """Convert a quaternion into a rotation matrix."""
#     qw, qz, qy, qx = quaternion
#     return np.array([
#         [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
#         [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
#         [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]
#     ])

def compute_relative_transformation_o(R1, t1, R2, t2):
    """Compute the transformation matrix of pose P2 relative to P1.
    
    Args:
    q1, t1: rotation matrix and translation (x, y, z) for pose P1.
    q2, t2: rotation matrix and translation (x, y, z) for pose P2.
    
    Returns:
    A 4x4 numpy array representing the transformation matrix of P2 relative to P1.
    """
    # Convert quaternions to rotation matrices

    
    # Create transformation matrices
    R1 = np.reshape(R1, (3, 3))
    R2 = np.reshape(R2, (3, 3))

    T1 = np.zeros((4, 4))
    T2 = np.zeros((4, 4))
    T1[:3, :3], T1[:3, 3], T1[3, 3] = R1, t1, 1
    T2[:3, :3], T2[:3, 3], T2[3, 3] = R2, t2, 1

    # print("T1: ", T1)
    # print("T2: ", T2)
    # print("R1: ", R1)
    # print("R2: ", R2)
    
    # Compute the inverse of T1
    T1_inv = np.linalg.inv(T1)
    
    # Compute the transformation of P2 relative to P1
    T2_1 = np.matmul(T1_inv, T2)
    
    return T2_1

def compute_relative_transformation(R1, t1, R2, t2):
    """Compute the transformation matrix of pose P2 relative to P1.
    
    Args:
    q1, t1: rotation matrix and translation (x, y, z) for pose P1.
    q2, t2: rotation matrix and translation (x, y, z) for pose P2.
    
    Returns:
    A 4x4 numpy array representing the transformation matrix of P2 relative to P1.
    """
    # Convert quaternions to rotation matrices

    
    # Create transformation matrices
    R1 = np.reshape(R1, (3, 3))
    R2 = np.reshape(R2, (3, 3))

    t_rel = t2 - t1

    T1_r = np.zeros((4, 4))
    T2_r = np.zeros((4, 4))
    T2_r_n = np.zeros((4, 4))
    T1_r[:3, :3], T1_r[:3, 3], T1_r[3, 3] = R1, np.array([0,0,0]), 1
    T2_r[:3, :3], T2_r[:3, 3], T2_r[3, 3] = R2, np.array([0,0,0]), 1

    T1 = np.eye(4)
    T2 = np.eye(4)
    T1[:3, :3], T1[:3, 3], T1[3, 3] = R1, t1, 1
    T2[:3, :3], T2[:3, 3], T2[3, 3] = R2, t2, 1

    # T1 = np.matmul(T1, T1_r)
    # T2 = np.matmul(T2, T2_r)

    # print("T1: ", T1)
    # print("T2: ", T2)
    # print("R1: ", R1)
    # print("R2: ", R2)
    
    # Compute the inverse of T1
    T1_inv = np.linalg.inv(T1)

    R1_inv = np.linalg.inv(R1)

    t2_n = np.dot(R1_inv, t_rel)
    
    # Compute the transformation of P2 relative to P1
    T2_1 = np.matmul(T1_inv, T2)

    T2_r_n[:3, :3], T2_r_n[:3, 3], T2_r_n[3, 3] = R2, t2_n, 1

    # print(T2_r_n)
    
    return T2_1

def rotation_matrix_to_quaternion(mat):
    """Convert a rotation matrix to a quaternion."""

    mat = mat.flatten()
    
    q = np.empty((4, ), dtype=np.float64)
    mujoco._functions.mju_mat2Quat(q, mat)
    return q

def quaternion_to_rotation_matrix(quaternion):
    
    """Convert a quaternion to rotation matrix."""

    quaternion = quaternion.flatten()
    
    mat = np.empty((9, ), dtype=np.float64)
    mujoco._functions.mju_quat2Mat(mat, quaternion)
    return mat.reshape((3, 3))

def recover_pose(T):
    """Recover quaternion and translation from a transformation matrix."""
    R = T[:3, :3]
    t = T[:3, 3]
    q = rotation_matrix_to_quaternion(R)
    return q, t


In [8]:
b_mat = env.data.geom_xmat[env.model.geom('arm-base-geom').id]
b_pos = env.data.geom_xpos[env.model.geom('arm-base-geom').id]
# b_pos = env.data.sensor("base-link-pos").data
# b_pos = env.data.sensor("base-link-pos").data
ee_ori = env.data.sensor("end-effector-orientation").data
ee_pos = env.data.sensor("end-effector-pos").data
ee_mat = quaternion_to_rotation_matrix(ee_ori)
# ee_mat = env.data.geom_xmat[env.model.geom('ee').id]
# ee_pos = env.data.geom_xpos[env.model.geom('ee').id]
mat = compute_relative_transformation(b_mat, b_pos, ee_mat, ee_pos)

tm_ori, tm_pos = recover_pose(mat)

# tm_pos = np.linalg.inv(b_mat.reshape(3,3)).dot(ee_mat.reshape(3,3)).dot(ee_pos)

print("tm_pos: ", tm_pos)
print("b_pos: ", b_pos)
print("ee_pos: ", ee_pos)
print("tm_ori: ", tm_ori)

print(np.linalg.norm(tm_pos - ee_pos))
print(np.linalg.norm(b_pos - ee_pos))
print(np.linalg.norm(b_pos- tm_pos))


tm_pos:  [0.14715246 0.02850137 1.21509131]
b_pos:  [0.02 0.   1.12]
ee_pos:  [0.16715246 0.02850137 2.33509131]
tm_ori:  [-0.01286256  0.97206374  0.11920746 -0.20178262]
1.1201785571952358
1.2243010477120344
0.16131470047255525


In [5]:
tm_mat = compute_relative_transformation(base_ori, base_pos, ee_ori, ee_pos)
print("tm_mat: ", tm_mat)

ValueError: cannot reshape array of size 4 into shape (3,3)

In [5]:
tm_ori, tm_pos = recover_pose(tm_mat)
print("tm_pos: ", tm_pos)
print("ee_pos: ", ee_pos)
print("ba_pos: ", base_pos)

tm_pos:  [ 0.98514421 -0.40568105 -0.00789449]
ee_pos:  [ 0.13331781 -0.00360452  1.09062497]
ba_pos:  [0.05066  0.000346 0.02841 ]


In [8]:
print(np.linalg.norm(tm_pos - base_pos))
print(np.linalg.norm(ee_pos - base_pos))

8.902995591711962
1.2211652827753392


In [36]:
for i in range(0, 10000):
    env.reset()
    env.step(np.zeros(6))

    ee_pos = env.data.sensor("end-effector-pos").data
    ee_ori = env.data.sensor("end-effector-orientation").data

    base_pos = env.data.sensor("base-link-pos").data
    base_ori = env.data.sensor("base-link-ori").data

    tm_ori, tm_pos = recover_pose(compute_relative_transformation(base_ori, base_pos, ee_ori, ee_pos))
    print(np.linalg.norm(tm_pos - base_pos))
    print(np.linalg.norm(ee_pos - base_pos))

0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.0206144006357825
0.23847620237087322
1.020

In [7]:
env.model.body('arm-base')

<_MjModelBodyViews
  dofadr: array([-1], dtype=int32)
  dofnum: array([0], dtype=int32)
  geomadr: array([1], dtype=int32)
  geomnum: array([1], dtype=int32)
  id: 1
  inertia: array([0.08631652, 0.08576911, 0.04921737])
  invweight0: array([0., 0.])
  ipos: array([0.05066 , 0.000346, 0.02841 ])
  iquat: array([ 0.41354962,  0.41836919, -0.57493859,  0.5686735 ])
  jntadr: array([-1], dtype=int32)
  jntnum: array([0], dtype=int32)
  mass: array([10.33])
  mocapid: array([-1], dtype=int32)
  name: 'arm-base'
  parentid: array([0], dtype=int32)
  pos: array([0., 0., 0.])
  quat: array([1., 0., 0., 0.])
  rootid: array([1], dtype=int32)
  sameframe: array([0], dtype=uint8)
  simple: array([0], dtype=uint8)
  subtreemass: array([17.316])
  user: array([], dtype=float64)
  weldid: array([0], dtype=int32)
>

In [6]:
b_mat = env.data.geom_xmat[env.model.geom('arm-base-geom').id]
b_pos = env.data.geom_xpos[env.model.geom('arm-base-geom').id]
# b_pos = env.data.sensor("base-link-pos").data
# b_pos = env.data.sensor("base-link-pos").data

ee_mat = env.data.geom_xmat[env.model.geom('ee').id]
ee_pos = env.data.geom_xpos[env.model.geom('ee').id]
mat = compute_relative_transformation(b_mat, b_pos, ee_mat, ee_pos)

tm_ori, tm_pos = recover_pose(mat)

# tm_pos = np.linalg.inv(b_mat.reshape(3,3)).dot(ee_mat.reshape(3,3)).dot(ee_pos)

print("tm_pos: ", tm_pos)
print("b_pos: ", b_pos)
print("ee_pos: ", ee_pos)
print("tm_ori: ", tm_ori)


[[ 0.9710668  -0.23709524 -0.02855014  0.08353756]
 [-0.23658993 -0.971404    0.01998701 -0.03201502]
 [-0.03247254 -0.01265405 -0.99939252  1.22804872]
 [ 0.          0.          0.          1.        ]]
tm_pos:  [ 0.08353756 -0.03201502  1.22804872]
b_pos:  [ 0.02  0.   10.12]
ee_pos:  [-2.88171157e-02  4.24662209e-03  1.13503276e+01]
tm_ori:  [-0.00822022  0.99270632 -0.11929137 -0.01536776]


In [6]:
print(np.linalg.norm(tm_pos - ee_pos))
print(np.linalg.norm(b_pos - ee_pos))
print(np.linalg.norm(b_pos- tm_pos))

10.120019762826551
1.2271381944317854
8.8984751691007


In [31]:
np.linalg.norm(mat[:3,:3].flatten() - ee_mat)

1.992532377290048

In [19]:
def mat2quat(mat):
    """Convert a rotation matrix to a quaternion."""
    q = np.empty((4, ), dtype=np.float64)
    mujoco._functions.mju_mat2Quat(q, mat)
    return q

a,b,c,d = mat2quat(mat_1)
a

0.7097371630972686

In [13]:
random_action = np.random.uniform(-1, 1, 6)
env.reset(play=False)
print(random_action)
for _ in range(10):
    # env.reset(play=False)
    env.step(random_action)

    ee_pos = env.data.sensor("end-effector-pos").data
    ee_ori = env.data.sensor("end-effector-orientation").data

    base_pos = env.data.sensor("base-link-pos").data
    base_ori = env.data.sensor("base-link-ori").data

    b_mat = env.data.geom_xmat[env.model.geom('arm-base-geom').id]
    b_pos = env.data.geom_xpos[env.model.geom('arm-base-geom').id]
    # b_pos = env.data.sensor("base-link-pos").data
    # b_pos = env.data.sensor("base-link-pos").data

    # ee_mat = env.data.geom_xmat[env.model.geom('ee').id]
    # ee_pos = env.data.geom_xpos[env.model.geom('ee').id]
    ee_ori = env.data.sensor("end-effector-orientation").data
    ee_pos = env.data.sensor("end-effector-pos").data
    ee_mat = quaternion_to_rotation_matrix(ee_ori)
    mat = compute_relative_transformation(b_mat.copy(), b_pos.copy(), ee_mat.copy(), ee_pos.copy())

    tm_ori, tm_pos = recover_pose(mat)

    # tm_pos = np.linalg.inv(b_mat.reshape(3,3)).dot(ee_mat.reshape(3,3)).dot(ee_pos)

    print("tm_pos rel: ", tm_pos.copy() - b_pos.copy())
    # print("b_pos: ", b_pos)
    print("ee_pos rel: ", ee_pos.copy() - b_pos.copy())
    # print("tm_ori: ", tm_ori)


[-0.09880347  0.73091946 -0.36653356  0.51341285 -0.42996564 -0.05384849]
tm_pos rel:  [0.12663962 0.00466235 0.0949126 ]
ee_pos rel:  [0.14663962 0.00466235 1.2149126 ]
tm_pos rel:  [ 0.38337051  0.01978312 -0.01597531]
ee_pos rel:  [0.40337051 0.01978312 1.10402469]
tm_pos rel:  [ 0.54880584  0.03477353 -0.16446149]
ee_pos rel:  [0.56880584 0.03477353 0.95553851]
tm_pos rel:  [ 0.61872866  0.04382048 -0.26396291]
ee_pos rel:  [0.63872866 0.04382048 0.85603709]
tm_pos rel:  [ 0.63815837  0.04778114 -0.30017491]
ee_pos rel:  [0.65815837 0.04778114 0.81982509]
tm_pos rel:  [ 0.63630895  0.04862587 -0.29817082]
ee_pos rel:  [0.65630895 0.04862587 0.82182918]
tm_pos rel:  [ 0.62762243  0.04803219 -0.28330925]
ee_pos rel:  [0.64762243 0.04803219 0.83669075]
tm_pos rel:  [ 0.61931668  0.04715    -0.26949304]
ee_pos rel:  [0.63931668 0.04715    0.85050696]
tm_pos rel:  [ 0.61410408  0.04651706 -0.26111424]
ee_pos rel:  [0.63410408 0.04651706 0.85888576]
tm_pos rel:  [ 0.611924    0.04622101 